In [7]:
import numpy as np
import pandas as pd
import math

def convert_x2_2_cate(column):
    lst = column.tolist()
    lst.sort()
    mapping = {}
    for i in lst[:6]:
        mapping[i] = 'low_x2'
    for i in lst[6:11]:
        mapping[i] = 'mid_x2'
    for i in lst[11:]:
        mapping[i] = 'high_x2'
    return column.map(mapping)

def convert_x3_2_cate(column):
    lst = column.tolist()
    lst.sort()
    mapping = {}
    for i in lst[:6]:
        mapping[i] = 'low_x3'
    for i in lst[6:11]:
        mapping[i] = 'mid_x3'
    for i in lst[11:]:
        mapping[i] = 'high_x3'
    return column.map(mapping)



class NodeA(object):
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.father = None
        self.children = []
    
    def split(self):
        if sum(self.label) == len(self.label) or sum(self.label) == 0:
            return
        min_split = None
        min_cost = None
        split_feat = None
        for i in self.data:
            category = set(self.data[i])
            node_list = []
            for j in category:
                node_list.append(NodeA(self.data[self.data[i] == j], self.label[self.data[i] == j]))
            # print(node_list)
            cost = sum([x.gini_index() * len(x.label) / len(self.label) for x in node_list])
            print(f'feat: {i} cost: {cost}\n')
            if min_cost is None or cost < min_cost:
                min_cost = cost
                min_split = node_list
                split_feat = i
        print(f'split with {split_feat}')

        print(f'cost: {min_cost}')
        for node in min_split:
            node.father = self
        self.children = min_split
        for node in min_split:
            print(node)


    def __repr__(self):
        return f'\n[{self.data}\n{self.label}]\n{self.gini_index()}\n'

    def gini_index(self):
        p1 = self.label.sum() / len(self.label)
        p2 = 1 - p1
        return 1 - p1**2 - p2**2


class NodeB(object):
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.father = None
        self.children = []
    
    def split(self):
        if sum(self.label) == len(self.label) or sum(self.label) == 0:
            return
        min_split = None
        min_cost = None
        split_feat = None
        split_value = None
        for i in self.data:
            # split with feature i
            # remaining category
            if i == 'x1' or i == 'x4':
                category = set(self.data[i])
                node_list = []
                for j in category:
                    node_list.append(NodeB(self.data[self.data[i] == j], self.label[self.data[i] == j]))
                # print(node_list)
                cost = sum([x.gini_index() * len(x.label) / len(self.label) for x in node_list])
                print(f'feat: {i} cost: {cost}\n')
                if min_cost is None or cost < min_cost:
                    min_cost = cost
                    min_split = node_list
                    split_feat = i
            else:
                
                values = self.data[i].tolist()
                values.sort()
                
                feat_min_cost = None
                feat_value = None
                for j in range(len(values) - 1):
                    node_list = []
                    node_list.append(NodeB(self.data[self.data[i] <= values[j]], self.label[self.data[i] <= values[j]]))
                    node_list.append(NodeB(self.data[self.data[i] > values[j]], self.label[self.data[i] > values[j]]))
                    cost = sum([x.gini_index() * len(x.label) / len(self.label) for x in node_list])
                    # print(f'feat: {i}, value: {values[j]}, cost: {cost}\n')
                    if min_cost is None or cost < min_cost:
                        min_cost = cost
                        min_split = node_list
                        split_feat = i
                        split_value = values[j]
                    if feat_min_cost is None or cost < feat_min_cost:
                        feat_min_cost = cost
                        feat_value = values[j]
                print(f'feat:{i}, feat min cost: {feat_min_cost}, split value: {feat_value}')
        print(f'split with {split_feat}')
        print(f'cost: {min_cost}')
        if split_feat in ['x2', 'x3']:
            print(f'split value {split_value}')

        for node in min_split:
            node.father = self
        self.children = min_split
        for node in min_split:
            print(node)


    def __repr__(self):
        return f'\n[{self.data}\n{self.label}]\n{self.gini_index()}\n'

    def gini_index(self):
        p1 = self.label.sum() / len(self.label)
        p2 = 1 - p1
        return 1 - p1**2 - p2**2


class NodeC(object):
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.father = None
        self.children = []
    
    def split(self):
        if sum(self.label) == len(self.label) or sum(self.label) == 0:
            return
        max_split = None
        max_IG = None
        split_feat = None
        split_value = None
        for i in self.data:
            # split with feature i
            # remaining category
            if i == 'x1' or i == 'x4':
                category = set(self.data[i])
                node_list = []
                for j in category:
                    node_list.append(NodeC(self.data[self.data[i] == j], self.label[self.data[i] == j]))
                # print(node_list)
                IG = self.entropy() - sum([x.entropy() * len(x.label) / len(self.label) for x in node_list])
                print(f'feat: {i} IG: {IG}\n')
                if max_IG is None or IG > max_IG:
                    max_IG = IG
                    max_split = node_list
                    split_feat = i
            else:
                
                values = self.data[i].tolist()
                values.sort()
                
                feat_max_IG = None
                feat_value = None
                for j in range(len(values) - 1):
                    node_list = []
                    node_list.append(NodeC(self.data[self.data[i] <= values[j]], self.label[self.data[i] <= values[j]]))
                    node_list.append(NodeC(self.data[self.data[i] > values[j]], self.label[self.data[i] > values[j]]))
                    IG = self.entropy() - sum([x.entropy() * len(x.label) / len(self.label) for x in node_list])
                    

                    if max_IG is None or IG > max_IG:
                        max_IG = IG
                        max_split = node_list
                        split_feat = i
                        split_value = values[j]
                    if feat_max_IG is None or IG > feat_max_IG:
                        feat_max_IG = IG
                        feat_value = values[j]

                    
                print(f'feat:{i}, feat max IG: {feat_max_IG}, split value: {feat_value}')
        print(f'split with {split_feat}')
        print(f'IG: {max_IG}')
        if split_feat in ['x2', 'x3']:
            print(f'split value {split_value}')

        for node in max_split:
            node.father = self
        self.children = max_split
        for node in max_split:
            print(node)


    def __repr__(self):
        return f'\n[{self.data}\n{self.label}]\n{self.entropy()}\n'

    def entropy(self):
        p1 = self.label.sum() / len(self.label)
        p2 = 1 - p1
        return - p1*math.log2(p1) if p1 != 0 else 0 - p2*math.log2(p2) if p2 != 0 else 0

class NodeD(object):
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.father = None
        self.children = []
    
    def split(self):
        if sum(self.label) == len(self.label) or sum(self.label) == 0:
            return
        max_split = None
        max_GR = None
        split_feat = None
        split_value = None
        for i in self.data:
            # split with feature i
            # remaining category
            if i == 'x1' or i == 'x4':
                category = set(self.data[i])
                node_list = []
                for j in category:
                    node_list.append(NodeD(self.data[self.data[i] == j], self.label[self.data[i] == j]))
                # print(node_list)
                if len(node_list) == 1:
                    continue
                IG = self.entropy() - sum([x.entropy() * len(x.label) / len(self.label) for x in node_list])
                GR = IG / -sum([len(node.label) / len(self.label) * math.log2(len(node.label)/len(self.label)) for node in node_list])
                print(f'feat: {i} GR: {GR}\n')
                if max_GR is None or GR > max_GR:
                    max_GR = GR
                    max_split = node_list
                    split_feat = i
            else:
                
                values = self.data[i].tolist()
                values.sort()
                
                feat_max_GR = None
                feat_value = None
                for j in range(len(values) - 1):
                    node_list = []
                    node_list.append(NodeD(self.data[self.data[i] <= values[j]], self.label[self.data[i] <= values[j]]))
                    node_list.append(NodeD(self.data[self.data[i] > values[j]], self.label[self.data[i] > values[j]]))
                    if len(node_list[0].label) == 0:
                        continue
                    if len(node_list[1].label) == 0:
                        continue
                    IG = self.entropy() - sum([x.entropy() * len(x.label) / len(self.label) for x in node_list])
                    GR = IG / -sum([len(node.label) / len(self.label) * math.log2(len(node.label)/len(self.label)) for node in node_list])

                    if max_GR is None or GR > max_GR:
                        max_GR = GR
                        max_split = node_list
                        split_feat = i
                        split_value = values[j]
                    if feat_max_GR is None or GR > feat_max_GR:
                        feat_max_GR = GR
                        feat_value = values[j]

                    
                print(f'feat:{i}, feat max GR: {feat_max_GR}, split value: {feat_value}')
        print(f'split with {split_feat}')
        print(f'GR: {max_GR}')
        if split_feat in ['x2', 'x3']:
            print(f'split value {split_value}')

        for node in max_split:
            node.father = self
        self.children = max_split
        for node in max_split:
            print(node)


    def __repr__(self):
        return f'\n[{self.data}\n{self.label}]\n{self.entropy()}\n'

    def entropy(self):
        if len(self.label) == 0:
            return 0
        p1 = self.label.sum() / len(self.label)
        p2 = 1 - p1
        return - p1*math.log2(p1) if p1 != 0 else 0 - p2*math.log2(p2) if p2 != 0 else 0




def do_split(node, depth):
    if depth == 2:
        return
    node.split()
    for child in node.children:
        do_split(child, depth+1)    





In [8]:
data = pd.read_csv('../data/Pr2_tree_dataset.csv')
    # print(data)
data['x2_cat'] = convert_x2_2_cate(data['x2'])
data['x3_cat'] = convert_x3_2_cate(data['x3'])
    # print(data)

root = NodeA(data.drop(['x2', 'x3', 'Class'], axis=1), data['Class'])
    # a.split()
do_split(root, 0)

feat: x1 cost: 0.34285714285714286

feat: x4 cost: 0.42857142857142855

feat: x2_cat cost: 0.45714285714285713

feat: x3_cat cost: 0.4047619047619047

split with x1
cost: 0.34285714285714286

[   x1 x4  x2_cat   x3_cat
3   Z  L  low_x2  high_x3
4   Z  L  low_x2   mid_x3
5   Z  W  low_x2   low_x3
9   Z  L  mid_x2   mid_x3
13  Z  W  low_x2   mid_x3
3     1
4     1
5     0
9     1
13    0
Name: Class, dtype: int64]
0.48


[   x1 x4   x2_cat   x3_cat
2   Y  L  high_x2   low_x3
6   Y  W   low_x2   low_x3
11  Y  W   mid_x2  high_x3
12  Y  L  high_x2   low_x3
2     1
6     1
11    1
12    1
Name: Class, dtype: int64]
0.0


[   x1 x4   x2_cat   x3_cat
0   X  L  high_x2   mid_x3
1   X  W   mid_x2  high_x3
7   X  L   mid_x2  high_x3
8   X  L   low_x2   low_x3
10  X  W   mid_x2   low_x3
0     0
1     0
7     0
8     1
10    1
Name: Class, dtype: int64]
0.48

feat: x1 cost: 0.48

feat: x4 cost: 0.0

feat: x2_cat cost: 0.4

feat: x3_cat cost: 0.26666666666666666

split with x4
cost: 0.0

[   x1 x4 

In [9]:
data = pd.read_csv('../data/Pr2_tree_dataset.csv')

root = NodeB(data.drop(['Class'], axis=1), data['Class'])
    # a.split()
do_split(root, 0)


feat: x1 cost: 0.34285714285714286

feat:x2, feat min cost: 0.3956043956043956, split value: 83
feat:x3, feat min cost: 0.3936507936507936, split value: 0.7
feat: x4 cost: 0.42857142857142855

split with x1
cost: 0.34285714285714286

[   x1  x2    x3 x4
3   Z  70  1.02  L
4   Z  68  0.70  L
5   Z  65  0.50  W
9   Z  75  0.70  L
13  Z  71  0.70  W
3     1
4     1
5     0
9     1
13    0
Name: Class, dtype: int64]
0.48


[   x1  x2    x3 x4
2   Y  83  0.66  L
6   Y  64  0.40  W
11  Y  72  0.90  W
12  Y  81  0.60  L
2     1
6     1
11    1
12    1
Name: Class, dtype: int64]
0.0


[   x1  x2   x3 x4
0   X  85  0.8  L
1   X  80  0.9  W
7   X  72  1.0  L
8   X  69  0.5  L
10  X  75  0.5  W
0     0
1     0
7     0
8     1
10    1
Name: Class, dtype: int64]
0.48

feat: x1 cost: 0.48

feat:x2, feat min cost: 0.3, split value: 65
feat:x3, feat min cost: 0.3, split value: 0.5
feat: x4 cost: 0.0

split with x4
cost: 0.0

[   x1  x2   x3 x4
5   Z  65  0.5  W
13  Z  71  0.7  W
5     0
13    0
Name: 

In [10]:
data = pd.read_csv('../data/Pr2_tree_dataset.csv')
root = NodeC(data.drop(['Class'], axis=1), data['Class'])
do_split(root, 0)

feat: x1 IG: 0.06300830809030594

feat:x2, feat max IG: 0.06873120251775766, split value: 83
feat:x3, feat max IG: 0.039644467147853135, split value: 0.7
feat: x4 IG: 0.017617449276040253

split with x2
IG: 0.06873120251775766
split value 83

[   x1  x2    x3 x4
1   X  80  0.90  W
2   Y  83  0.66  L
3   Z  70  1.02  L
4   Z  68  0.70  L
5   Z  65  0.50  W
6   Y  64  0.40  W
7   X  72  1.00  L
8   X  69  0.50  L
9   Z  75  0.70  L
10  X  75  0.50  W
11  Y  72  0.90  W
12  Y  81  0.60  L
13  Z  71  0.70  W
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    0
Name: Class, dtype: int64]
0.36727941925300145


[  x1  x2   x3 x4
0  X  85  0.8  L
0    0
Name: Class, dtype: int64]
0.0

feat: x1 IG: 0.04336428213772309

feat:x2, feat max IG: 0.01616119801778204, split value: 80
feat:x3, feat max IG: 0.01820322266123542, split value: 0.7
feat: x4 IG: 0.033867532482333096

split with x1
IG: 0.04336428213772309

[   x1  x2    x3 x4
3   Z  70  1.02 

In [11]:
data = pd.read_csv('../data/Pr2_tree_dataset.csv')
root = NodeD(data.drop(['Class'], axis=1), data['Class'])
do_split(root, 0)

feat: x1 GR: 0.03994424821002434

feat:x2, feat max GR: 0.18514336598775558, split value: 83
feat:x3, feat max GR: 0.04216213884965609, split value: 0.7
feat: x4 GR: 0.017881593746352144

split with x2
GR: 0.18514336598775558
split value 83

[   x1  x2    x3 x4
1   X  80  0.90  W
2   Y  83  0.66  L
3   Z  70  1.02  L
4   Z  68  0.70  L
5   Z  65  0.50  W
6   Y  64  0.40  W
7   X  72  1.00  L
8   X  69  0.50  L
9   Z  75  0.70  L
10  X  75  0.50  W
11  Y  72  0.90  W
12  Y  81  0.60  L
13  Z  71  0.70  W
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    0
Name: Class, dtype: int64]
0.36727941925300145


[  x1  x2   x3 x4
0  X  85  0.8  L
0    0
Name: Class, dtype: int64]
0.0

feat: x1 GR: 0.027504565830191344

feat:x2, feat max GR: 0.02609244850211408, split value: 80
feat:x3, feat max GR: 0.020441767040899596, split value: 0.7
feat: x4 GR: 0.03401285403090858

split with x4
GR: 0.03401285403090858

[   x1  x2   x3 x4
1   X  80  0.9  W